In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import threading
import queue
import time
from pymodbus.client import ModbusTcpClient

In [2]:
cam = 2
fps = 240 #Maximum achievable was 163.06

model_version = "my_model_20.0.keras"
max_frames = 10  # Number of frames to evaluate

image_path = "no_bottle.jpg"
frame_buffer = []
raw_frame_queue = queue.Queue(maxsize=20)  # Queue for raw frames
processed_frame_queue = queue.Queue(maxsize=10)  # Queue for processed frames

Im_Width = 128
Im_Height = 128

#crop
x = 120
y = 0
w = 400
h = 200

a = 290
b = 110
c = 100
d = 100

max_brightness = 175

stop_event = threading.Event()

coil_address = 1536 #ModBus Y0
PLC_IP = "192.168.1.111"  #PLC's IP address
PLC_PORT = 502           # Default Modbus TCP port

In [3]:
def modbus(PLC_IP, PLC_PORT):

    # Create Modbus client
    client = ModbusTcpClient(PLC_IP, port=PLC_PORT)

    # Connect to the PLC
    if client.connect():
        print("Connection successful")
    else:
        print("Failed to connect to PLC")
    return client

In [4]:
def PLC(client, coil_address,value):
    write_result = client.write_coil(address=coil_address, value=value)  # Write 'True' to turn ON
    if write_result.isError():
        print("Write Error:", write_result)
    

In [5]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [6]:
def measure_brightness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [7]:
def measure_edge_richness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    edges = cv2.Canny(gray, 100, 200)  # Apply Canny edge detection
    edge_richness = np.sum(edges)  # Sum of edge pixel intensities
    return edge_richness

In [8]:
def select_best_frame(frame_buffer, x, y, w, h, max_brightness):

    best_frame = None
    highest_brightness = 0
    highest_edge_richness = 400000

    for frame in frame_buffer:

        #Crop
        cropped_frame = crop_image(frame, x, y, w, h)

        # Measure brightness
        brightness = measure_brightness(cropped_frame)

        if brightness > max_brightness :
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame
        
        elif brightness > 150 and brightness < 176:
            # Apply edge detection
            edge_richness = measure_edge_richness(cropped_frame)
            # Update the best frame if this one is better
            if edge_richness > highest_edge_richness:
                highest_edge_richness = edge_richness
                best_frame = frame  # Save the cropped frame as the best frame


    return best_frame  # Returns None if no frames meet the threshold


In [9]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [10]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

   # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        PLC(client, coil_address,False)
        return "Non-Defective", prediction       
        
    else:
        PLC(client, coil_address,True)
        return "Defective", prediction

In [11]:
def capture(cam, fps, frame_buffer, raw_frame_queue, max_frames, stop_event):
    
    cap = cv2.VideoCapture(cam)  # Open webcam
    cap.set(cv2.CAP_PROP_FPS, fps)  # Set the desired FPS

    if not cap.isOpened():
        print("Error: Cannot open camera.")
        return

    while not stop_event.is_set():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        frame_buffer.append(frame)

        # Yield the buffer when it's full
        if len(frame_buffer) == max_frames:
            raw_frame_queue.put(frame_buffer.copy())  # Add a copy of the buffer
            frame_buffer.clear()  # Clear the buffer for the next batch

    cap.release()

In [12]:
def selector(raw_frame_queue, processed_frame_queue, a, b, c, d, stop_event):
    while not stop_event.is_set() or not raw_frame_queue.empty():
        try:
            frame_buffer = raw_frame_queue.get(timeout=1)
        except queue.Empty:
            continue

        processed_buffer = []
        for frame in frame_buffer:
            cropped_frame = crop_image(frame, a, b, c, d)
            brightness = measure_brightness(cropped_frame)

            if not (brightness > 60 and brightness < 65):
                processed_buffer.append(frame)  # Pass frames meeting the condition

        if processed_buffer:
            try:
                processed_frame_queue.put(processed_buffer, timeout=1)
            except queue.Full:
                print("Processed frame queue is full. Dropping buffer.")

    print("Processing thread stopped.")


In [13]:
def runModel(model_version, processed_frame_queue, x, y, w, h, max_brightness, stop_event, target_size=(Im_Width, Im_Height)):

    # Load the model
    print("Loading model...") 
    model = load_model(model_version)
    print("Model loaded successfully.")
    
    while True:
        frame_buffer = processed_frame_queue.get()
        best_frame = select_best_frame(frame_buffer, x, y, w, h, max_brightness)
        # Process and predict
        if best_frame is not None:
            feedback, confidence = predict_frame(best_frame, model, target_size)
            

            prob = abs(round(confidence,2)- 0.5)*200
            print(f"Prediction: {feedback} (Confidence: {prob:.2f})")

            # Display feedback on the frame
            cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(best_frame, f"Confidence: {prob:.0f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Display the best frame
            cv2.imshow("Best Frame", best_frame)

        else:
            # Load and display the "no-bottle" image in the same window
            PLC(client, coil_address,False)
            image = cv2.imread(image_path)
            cv2.putText(image, "No Bottle!", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Best Frame", image)

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            #run = False
            stop_event.set()
            break

    cv2.destroyAllWindows()
    #return run

In [14]:
capture_thread = threading.Thread(target=capture, args=(cam, fps, frame_buffer, raw_frame_queue, max_frames, stop_event))
selector_thread = threading.Thread(target=selector, args=(raw_frame_queue, processed_frame_queue, a, b, c, d, stop_event))
runModel_thread = threading.Thread(target=runModel, args=(model_version, processed_frame_queue, x, y, w, h, max_brightness, stop_event))

In [15]:
if __name__ == "__main__":

    client = modbus(PLC_IP, PLC_PORT)

    # Start the threads
    capture_thread.start()
    time.sleep(0.1)
    selector_thread.start()
    time.sleep(0.1)
    runModel_thread.start()

    # Wait for threads to complete
    capture_thread.join()
    selector_thread.join()
    #frame_queue.join()
    runModel_thread.join()


Connection successful


Loading model...
Model loaded successfully.
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 1536)
Output Y0 turned ON (Coil 15